In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from joblib import dump

# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Load your data
categories_path = '/Users/rishabhtiwari/Desktop/training_dataset/categories.npy'
valid_periods_path = '/Users/rishabhtiwari/Desktop/training_dataset/valid_periods.npy'
training_data_path = '/Users/rishabhtiwari/Desktop/training_dataset/training_data.npy'

categories = np.load(categories_path)
valid_periods = np.load(valid_periods_path)
training_data = np.load(training_data_path)

In [ ]:
# Constants for sequence and forecast lengths
seq_length = 128
forecast_length = 9

# Data preprocessing
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [ ]:
# Function to preprocess data and create sequences
def preprocess_data(data, valid_periods, seq_length, forecast_length, scaler_X, scaler_y):
    X, y = [], []
    for i, row in enumerate(data):
        valid_data = row[valid_periods[i][0]:valid_periods[i][1]]
        if valid_data.size > 0:
            for j in range(len(valid_data) - seq_length - forecast_length + 1):
                seq_X = valid_data[j:(j + seq_length)]
                seq_y = valid_data[(j + seq_length):(j + seq_length + forecast_length)]
                X.append(seq_X)
                y.append(seq_y)
    
    X = np.array(X).reshape(-1, seq_length)
    y = np.array(y).reshape(-1, forecast_length)
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    return X, y

In [ ]:
# Apply preprocessing
X, y = preprocess_data(training_data, valid_periods, seq_length, forecast_length, scaler_X, scaler_y)

In [ ]:
# Save the fitted scalers
scaler_X_filename = "/mnt/data/scaler_X.save"
scaler_y_filename = "/mnt/data/scaler_y.save"
dump(scaler_X, scaler_X_filename)
dump(scaler_y, scaler_y_filename)

In [ ]:
# Splitting the data
val_size = int(len(X) * 0.2)
indices = np.arange(len(X))
np.random.shuffle(indices)
X_train, y_train = X[indices[:-val_size]], y[indices[:-val_size]]
X_val, y_val = X[indices[-val_size:]], y[indices[-val_size:]]

In [ ]:
# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], seq_length, 1))
y_train = y_train.reshape((y_train.shape[0], forecast_length, 1))
X_val = X_val.reshape((X_val.shape[0], seq_length, 1))
y_val = y_val.reshape((y_val.shape[0], forecast_length, 1))

In [ ]:
# Model building
def build_advanced_model(input_shape, lstm_units, dropout_rate, forecast_length):
    input_layer = Input(shape=input_shape)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True))(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = LSTM(lstm_units // 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(forecast_length)(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

input_shape = (seq_length, 1)
dropout_rate = 0.2
lstm_units = 128

model = build_advanced_model(input_shape, lstm_units, dropout_rate, forecast_length)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Model training
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=1)

In [ ]:
# Save the model
model_save_path = '/mnt/data/SubmissionModel'
model.save(model_save_path)